In [1]:
%pip install ipympl


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /usr/local/bin/python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
#!/usr/bin/env python3
# Minimal RPY reader for MuJoCo model "monstertruck.xml"

import math
import mujoco as mj
from pathlib import Path

def quat_wxyz_to_rpy(qw, qx, qy, qz):
    # ZYX convention (yaw-pitch-roll)
    R00 = 1 - 2*(qy*qy + qz*qz)
    R10 = 2*(qx*qy + qw*qz)
    R20 = 2*(qx*qz - qw*qy)
    R21 = 2*(qy*qz + qw*qx)
    R22 = 1 - 2*(qx*qx + qy*qy)
    yaw   = math.atan2(R10, R00)
    pitch = -math.asin(max(-1.0, min(1.0, R20)))
    roll  = math.atan2(R21, R22)
    return roll, pitch, yaw

# Load model & data from file
file_name = "monstertruck.xml"
PROJECT_ROOT = Path.cwd()
xml_path = PROJECT_ROOT.parent / "model" / file_name
m = mj.MjModel.from_xml_path(str(xml_path))
d = mj.MjData(m)
mj.mj_resetData(m, d)   # ensure qpos initialized from XML defaults

# Find the first free joint (the chassis) and get its quaternion
free_j = next(j for j in range(m.njnt) if m.jnt_type[j] == mj.mjtJoint.mjJNT_FREE)
adr = m.jnt_qposadr[free_j]  # qpos layout: [x y z qw qx qy qz]
qw, qx, qy, qz = d.qpos[adr+3:adr+7]

roll, pitch, yaw = quat_wxyz_to_rpy(qw, qx, qy, qz)
print(f"{roll:.6f} {pitch:.6f} {yaw:.6f}")


3.141593 -0.000000 0.000000


In [7]:
# Second cell — viewer 300s with random throttle updates [-1, 1]
import time, random
import mujoco as mj
import mujoco.viewer as viewer

# chassis quaternion indices [qw qx qy qz] in qpos
free_j = next(j for j in range(m.njnt) if m.jnt_type[j] == mj.mjtJoint.mjJNT_FREE)
qadr = m.jnt_qposadr[free_j] + 3

DURATION   = 300.0   # seconds to run
CMD_PERIOD = 0.5     # change throttle every 0.5 s
#LOG_PERIOD = 0.5     # print every 0.5 s
dt = float(m.opt.timestep)

# Reset to initial XML state before running
d = mj.MjData(m)        # fresh data buffer
mj.mj_resetData(m, d)   # load XML defaults (incl. chassis pos/quat)
mj.mj_forward(m, d)     # recompute kinematics/contacts
d.ctrl[:] = 0.0         # clear any leftover control

with viewer.launch_passive(m, d) as v:
    t0 = d.time
    next_cmd = t0
    next_log = t0
    while d.time - t0 < DURATION:
        # update random throttle on schedule
        if d.time >= next_cmd:
            u = random.uniform(-1.0, 1.0)
            d.ctrl[:] = u  # same throttle to all 4 motors
            next_cmd += CMD_PERIOD

        mj.mj_step(m, d)

        # optional: print RPY + current throttle
        if d.time >= next_log:
            roll, pitch, yaw = quat_wxyz_to_rpy(*d.qpos[qadr:qadr+4])
            print(f"t={d.time - t0:6.2f}s  u={u:+.2f}  rpy=(roll: {roll:+.3f}, pitch: {pitch:+.3f}, yaw: {yaw:+.3f})", end="\r")
            #next_log += LOG_PERIOD

        v.sync()
        time.sleep(dt)

print("\nDone.")


RuntimeError: `launch_passive` requires that the Python script be run under `mjpython` on macOS